In [1]:
from elexon import ElexonRawClient
from calendar import monthrange
import datetime
import pandas as pd
from tabulate import tabulate
from tqdm import tqdm
from sys import exit

In [2]:
# Large Volume and Price
# Market Index Price and Volume.
# System Buy - Price and Volume.

In [3]:
api = ElexonRawClient(api_key='syfypqvxf8hb7b9',api_service_type ='xml') # available for free from the Elexon Portalprint

In [4]:
# Year = 2019
# Month = '03'
# Day = '31'
# Per = 46
# Date = f"{Year}-{Month}-{Day}"
# ImbalancePrice, ImbalanceVolume = None, None
# try:
#     ImbalancePrice = api.request('B1770',SettlementDate=Date,Period=Per)
#     ImbalanceVolume = api.request('B1780',SettlementDate=Date,Period=Per)
#     ImbPrice = ImbalancePrice[0]['imbalancePriceAmountGBP'] # System Buy/Sell Price
#     ImbVolume = ImbalanceVolume[0]['imbalanceQuantityMAW'] # System Buy/Sell Volume MWh
#     print(f"{ImbPrice} £/MWh {ImbVolume} MWh.")
# except:
#     pass

32.6 £/MWh -203.1194 MWh.
[{'timeSeriesID': 'ELX-EMFIP-IMBP-TS-1', 'businessType': 'Balance energy deviation', 'controlArea': '10YGB----------A', 'settlementDate': datetime.date(2019, 3, 31), 'settlementPeriod': 46, 'imbalancePriceAmountGBP': 32.6, 'priceCategory': 'Insufficient balance', 'curveType': 'Sequential fixed size block', 'resolution': 'PT30M', 'documentType': 'Imbalance prices', 'processType': 'Realised', 'activeFlag': True, 'docStatus': 'Final', 'documentID': 'ELX-EMFIP-IMBP-20572247', 'documentRevNum': 1}, {'timeSeriesID': 'ELX-EMFIP-IMBP-TS-2', 'businessType': 'Balance energy deviation', 'controlArea': '10YGB----------A', 'settlementDate': datetime.date(2019, 3, 31), 'settlementPeriod': 46, 'imbalancePriceAmountGBP': 32.6, 'priceCategory': 'Excess balance', 'curveType': 'Sequential fixed size block', 'resolution': 'PT30M', 'documentType': 'Imbalance prices', 'processType': 'Realised', 'activeFlag': True, 'docStatus': 'Final', 'documentID': 'ELX-EMFIP-IMBP-20572247', 'docu

In [7]:
SystemDataTable = pd.DataFrame(columns=['Settlement Date','Settlement Period','System Sell Price(GBP/MWh)','Net Imbalance Volume(MWh)'])
Years = tqdm(range(2014,2024+1))
for Year in Years:
    for Month in range(1,12+1):
        Days = monthrange(Year,Month)[1]
        for Day in range(1,Days+1):
            MonthStr = str(Month).zfill(2)
            DayStr = str(Day).zfill(2)
            DateString = f"{Year}-{MonthStr}-{DayStr}"
            Date = f"{Day}/{Month}/{Year}"
            for Per in range(1,50+1):
                PerStr = str(Per).zfill(2)
                Years.set_description(f'{Year}-{MonthStr}-{DayStr} ## Per {PerStr}')
                DataList = []
                ImbalancePrice, ImbalanceVolume = None, None
                try:
                    ImbalancePrice = api.request('B1770',SettlementDate=DateString,Period=Per)
                    ImbalanceVolume = api.request('B1780',SettlementDate=DateString,Period=Per)
                    ImbPrice = ImbalancePrice[0]['imbalancePriceAmountGBP'] # System Buy/Sell Price
                    ImbVolume = ImbalanceVolume[0]['imbalanceQuantityMAW'] # System Buy/Sell Volume MWh
                    DataList.extend([Date,Per,ImbPrice,ImbVolume])
                    SystemDataTable.loc[len(SystemDataTable), SystemDataTable.columns] = (DataList)
                except:
                    pass
SystemDataTable.to_csv("SystemBuySellData2024.csv",index=False)

2024-03-31 ## Per 50: 100%|██████████| 1/1 [09:17<00:00, 557.37s/it]
